In [0]:
# ===============================================================
# 🧪 Step 8 Validation Cell — "Deploy Your Code for Testing"
# ---------------------------------------------------------------
# Purpose : Lightweight runtime validation of configuration, Spark,
#            and basic pipeline functionality prior to production deploy.
# Runtime : Databricks 16.4 LTS (Unity Catalog)
# Author  : M. Holahan
# Date    : 2025-11-04
# ===============================================================

# COMMAND ----------
# ✅ Environment Bootstrap

import sys, os, json, importlib
from pyspark.sql import SparkSession, Row

# Mirror Pipeline_Runner import pattern
sys.path.append("/Workspace/Users/.../scripts")  # adjust to your workspace path
config = importlib.import_module("scripts.config")

# Build CONFIG dict dynamically from uppercase vars in config.py
CONFIG = {
    k: getattr(config, k)
    for k in dir(config)
    if k.isupper() and not k.startswith("__")
}

# COMMAND ----------
# 🔍 1️⃣ Config Validation — simplified and bulletproof
print("🔍 Checking config file integrity...")

try:
    print("Detected keys:", list(CONFIG.keys())[:10], "...")
    print(f"✅ Config imported successfully from: {config.__file__}")
    print(json.dumps(CONFIG, indent=2))
except Exception as e:
    print("❌ Config validation failed:", e)
    raise


# COMMAND ----------
# ⚙️ 2️⃣ Spark Session Validation — ultra-simple and resilient

print("\n🚀 Initializing Spark session...")

try:
    app_name = CONFIG.get("SPARK_APP_NAME", "UnguidedCapstoneTest")  # fallback name
    spark = SparkSession.builder.appName(app_name).getOrCreate()

    print("✅ Spark session active.")
    print("Spark version:", spark.version)
    print("App name:", spark.sparkContext.appName)
    print("Master:", spark.sparkContext.master)
except Exception as e:
    print("❌ Spark session test failed:", e)
    raise


# COMMAND ----------
# 📊 3️⃣ DataFrame Operation Test

print("\n📊 Running basic Spark DataFrame test...")

try:
    df = spark.createDataFrame([Row(id=1, name="Test"), Row(id=2, name="Validation")])
    count = df.count()
    assert count == 2
    print("✅ DataFrame test passed — row count:", count)
    df.show()
except Exception as e:
    print("❌ DataFrame validation failed:", e)
    raise

# COMMAND ----------
# 🔐 4️⃣ Environment Variable Check

print("\n🔐 Checking for optional API keys...")
for key in ["TMDB_API_KEY", "DISCOGS_TOKEN"]:
    print(f"{'✅' if os.getenv(key) else '⚠️'} {key} {'found' if os.getenv(key) else 'not found'}")

# COMMAND ----------
# 📂 5️⃣ Directory Existence Check

print("\n🧩 Checking data directories...")
paths = [
    CONFIG["DATA_DIR"],
    "/Workspace/Users/.../data/interim",
    "/Workspace/Users/.../data/processed"
]
for p in paths:
    print(f"{'✅' if os.path.exists(p) else '⚠️'} Path check: {p}")

# COMMAND ----------
# 🎯 6️⃣ Summary Report

print("\n🎉 Step 8 Testing Complete!")
print("---------------------------------------------------")
print("✅ Config module imported and verified")
print("✅ Spark session initialized successfully")
print("✅ DataFrame test executed correctly")
print("✅ Environment and paths checked")
print("---------------------------------------------------")
print("All tests PASSED — Ready for Step 9 Deployment 🚀")
